In [1]:
import torch
from datasets import load_dataset
from transformers import RobertaTokenizer, RobertaForCausalLM, Trainer, TrainingArguments
torch.cuda.empty_cache()


c:\Users\sainithin\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [3]:
dataset = load_dataset("code_search_net", "python",cache_dir="./Datasets", trust_remote_code=True)

In [4]:
dataset.column_names

{'train': ['repository_name',
  'func_path_in_repository',
  'func_name',
  'whole_func_string',
  'language',
  'func_code_string',
  'func_code_tokens',
  'func_documentation_string',
  'func_documentation_tokens',
  'split_name',
  'func_code_url'],
 'test': ['repository_name',
  'func_path_in_repository',
  'func_name',
  'whole_func_string',
  'language',
  'func_code_string',
  'func_code_tokens',
  'func_documentation_string',
  'func_documentation_tokens',
  'split_name',
  'func_code_url'],
 'validation': ['repository_name',
  'func_path_in_repository',
  'func_name',
  'whole_func_string',
  'language',
  'func_code_string',
  'func_code_tokens',
  'func_documentation_string',
  'func_documentation_tokens',
  'split_name',
  'func_code_url']}

In [5]:
tokenizer = RobertaTokenizer.from_pretrained("microsoft/codebert-base", cache_dir="./Models")
model = RobertaForCausalLM.from_pretrained("microsoft/codebert-base", cache_dir="./Models",is_decoder = True).to(device)

Some weights of RobertaForCausalLM were not initialized from the model checkpoint at microsoft/codebert-base and are newly initialized: ['lm_head.bias', 'lm_head.decoder.bias', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
def tokenize_code(code_snippet, tokenizer):
    return tokenizer(
        code_snippet, max_length=512, truncation=True, padding='max_length', return_tensors="pt"
    )

In [7]:
def preprocess_data(batch, tokenizer):
    batch["input_ids"] = tokenize_code(batch["func_code_string"], tokenizer)["input_ids"]
    batch["labels"] = tokenize_code(batch["func_documentation_string"], tokenizer)["input_ids"]
    return batch

In [8]:
train_dataset = dataset["train"].shuffle(seed=42).select(range(10000)).map(lambda x: preprocess_data(x, tokenizer), batched=True)
eval_dataset = dataset["validation"].shuffle(seed=42).select(range(3000)).map(lambda x: preprocess_data(x, tokenizer), batched=True)

In [9]:
model

RobertaForCausalLM(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNor

In [10]:
training_args = TrainingArguments(
    output_dir="./codebert-finetuned-roberta/results",    # output directory
    num_train_epochs=20,                   # number of epochs
    per_device_train_batch_size=4,       # batch size
    per_device_eval_batch_size=4,        # eval batch size
    warmup_steps=1000,                     # warmup steps
    weight_decay=0.01,                    # weight decay
    logging_dir="./logs",                 # logging directory
    logging_steps=1000,
    evaluation_strategy="steps",          # Evaluation after every logging step
    save_total_limit=2,                   # Keep only last two checkpoints
    save_steps=1000,                       # Save model every 500 steps
    report_to="none",
    fp16=True,
    learning_rate=5e-5                 # No reports (e.g., to wandb)
)

c:\Users\sainithin\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [11]:
# Trainer setup
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
)

In [12]:
trainer.train()

  0%|          | 0/18750 [00:00<?, ?it/s]We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.
c:\Users\sainithin\AppData\Local\Programs\Python\Python312\Lib\site-packages\torch\autograd\graph.py:825: UserWarning: cuDNN SDPA backward got grad_output.strides() != output.strides(), attempting to materialize a grad_output with matching strides... (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\cudnn\MHA.cpp:676.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass
  1%|          | 100/18750 [00:29<1:29:32,  3.47it/s]

{'loss': 13.6835, 'grad_norm': 41.94532012939453, 'learning_rate': 4.800000000000001e-06, 'epoch': 0.08}


                                                     
  1%|          | 100/18750 [00:47<1:29:32,  3.47it/s]

{'eval_loss': 5.897005558013916, 'eval_runtime': 17.3952, 'eval_samples_per_second': 57.487, 'eval_steps_per_second': 14.372, 'epoch': 0.08}


  1%|          | 200/18750 [01:16<1:28:57,  3.48it/s] 

...

                                                       
100%|█████████▉| 18700/18750 [2:27:11<00:14,  3.39it/s]

{'eval_loss': 1.6356921195983887, 'eval_runtime': 17.6319, 'eval_samples_per_second': 56.715, 'eval_steps_per_second': 14.179, 'epoch': 14.96}


100%|██████████| 18750/18750 [2:27:27<00:00,  2.12it/s]

{'train_runtime': 8847.7985, 'train_samples_per_second': 8.477, 'train_steps_per_second': 2.119, 'train_loss': 1.0105573541259765, 'epoch': 15.0}


TrainOutput(global_step=18750, training_loss=1.0105573541259765, metrics={'train_runtime': 8847.7985, 'train_samples_per_second': 8.477, 'train_steps_per_second': 2.119, 'total_flos': 1.97449097472e+16, 'train_loss': 1.0105573541259765, 'epoch': 15.0})

In [13]:
model.save_pretrained("./codebert-finetuned-roberta")
tokenizer.save_pretrained("./codebert-finetuned-roberta")

('./codebert-finetuned-roberta\\tokenizer_config.json',
 './codebert-finetuned-roberta\\special_tokens_map.json',
 './codebert-finetuned-roberta\\vocab.json',
 './codebert-finetuned-roberta\\merges.txt',
 './codebert-finetuned-roberta\\added_tokens.json')

In [14]:
def generate_documentation(code_snippet, model, tokenizer):
    # Tokenize input
    inputs = tokenizer(code_snippet, return_tensors="pt", padding=True, truncation=True).to(device)
    
    # Generate output
    outputs = model.generate(inputs["input_ids"], max_length=150, num_beams=5, early_stopping=True)
    
    # Decode generated text
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

In [15]:
code_example = """
def add_numbers(a, b):
    return a + c
"""

In [16]:
documentation = generate_documentation(code_example, model, tokenizer)
print("Generated Documentation:", documentation)

Generated Documentation: 
def add_numbers(a, b):
    return a + c



In [17]:
model

RobertaForCausalLM(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNor